In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
import copy

palavras_mais_usadas = 10000
tamanho_maximo = 500
tamanho_da_camada_de_incorporacao = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=palavras_mais_usadas)

x_train = sequence.pad_sequences(x_train,tamanho_maximo)
x_test = sequence.pad_sequences(x_test,tamanho_maximo)

Using TensorFlow backend.


In [2]:
json_dicionario = imdb.get_word_index()
dicionarioPalavras = list(json_dicionario.keys())
dicionarioNumeros = list(json_dicionario.values())
def traduzir(revisao):
    retorno = []
    for palavra in revisao:
        if(palavra == 0):
            retorno.append('')
        elif(palavra == 1):
            retorno.append('>')
        elif(palavra == 2):
            retorno.append('?')
        else:
            index = dicionarioNumeros.index(palavra - 3)
            retorno.append(dicionarioPalavras[index])
    return ' '.join(retorno)  

In [3]:
model = Sequential()
model.add(Embedding(palavras_mais_usadas, tamanho_da_camada_de_incorporacao, input_length = tamanho_maximo))
model.add(Flatten())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                256016    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 576,305
Trainable params: 576,305
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
treino = model.fit(x_train,y_train,validation_data = (x_test,y_test), epochs=3, batch_size=128 )

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 13s 537us/step - loss: 0.5974 - accuracy: 0.6208 - val_loss: 0.3250 - val_accuracy: 0.8617
Epoch 2/3
19200/25000 [======================>.......] - ETA: 2s - loss: 0.2271 - accuracy: 0.9118

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline
precisao = treino.history['accuracy']
validacao = treino.history['val_accuracy']
epochs = range(1, len(precisao) + 1)
plt.plot(epochs, precisao, '-', label='Precisão do treinamento')
plt.plot(epochs, validacao, ':', label='Precisão da validação')
plt.title('Precisão do treinamento e da validação')
plt.xlabel('Época')
plt.ylabel('Precisão')
plt.legend(loc='upper left')
plt.plot()

In [ ]:
scores = model.evaluate(x_test,y_test,verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

In [ ]:
import string
import numpy as np
def analise(texto):
    tabela = str.maketrans("","", string.punctuation)
    traducao = texto.translate(tabela)
    traducao = traducao.lower().split(" ")
    traducao = [palavra for palavra in traducao if palavra.isalpha()]

    
    critica = []
    critica.append(1)
    for palavra in traducao:
        if palavra in json_dicionario and json_dicionario[palavra] <= palavras_mais_usadas:
            critica.append(json_dicionario[palavra])
        else:
            critica.append(2)
    padded_input = sequence.pad_sequences([critica], maxlen=tamanho_maximo)
    result = model.predict(np.array([padded_input][0]))[0][0]
    return result
print(analise("this movie was wonderful."))
print(analise('Easily the most stellar experience I have ever had.'))
print(analise('The long lines and poor customer service really turned me off.'))